In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import warnings
warnings.filterwarnings('ignore')

# all import statements
import numpy as np
import pandas as pd
import pydicom as pyd
import os
import matplotlib.pyplot as plt
import mudicom

from os.path import dirname, join
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from PIL import Image

from scipy.misc import imresize
from scipy.signal import convolve2d
import scipy
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

from numpy import newaxis

from skimage.measure import shannon_entropy
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy


import math
from skimage.measure import label, regionprops

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import matplotlib.patches as mpatches

import statistics

from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

import cv2
from skimage import img_as_float
from skimage import exposure

In [3]:
import pickle

from tqdm import tqdm
import os
import sys

In [4]:
def load_pickle(fileName):
    with open(fileName, "rb") as fp:
        file = pickle.load(fp)
    return file 

def write_pickle(file, fileName):
    with open(fileName, "wb") as fp:
        pickle.dump(file, fp)

### GLCM

In [5]:

def s_entropy(dcm):
    return shannon_entropy(dcm)

def entropy_simple(dcm):
    return entropy(dcm)

class glcm:
    def __init__(self, image):
        distance = [1, 2, 3]
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
        self.image = img_as_ubyte(image.astype('int64'))
        self.glcm_mat = greycomatrix(self.image, distances = distance, angles = angles, symmetric = True, normed = True)
        self.properties = ['correlation', 'homogeneity', 'contrast', 'energy']
            
    def correlation(self):
        return greycoprops(self.glcm_mat, 'correlation').flatten()
    
    def homogeneity(self):
        return greycoprops(self.glcm_mat, 'homogeneity').flatten()
    
    def contrast(self):
        return greycoprops(self.glcm_mat, 'contrast').flatten()
    
    def energy(self):
        return greycoprops(self.glcm_mat, 'energy').flatten()
    
    def glcm_all(self):
        return np.hstack([greycoprops(self.glcm_mat, props).ravel() for props in self.properties])
    

### Region Props

In [6]:
# thresh_ada = threshold_otsu(image_adahist)
# bw_ada = closing(image_adahist > thresh_ada, square(3))
# bw_clear_ada = clear_border(bw_ada)

class region_props:
    def __init__(self, image):
        self.image = image
        self.thresh = threshold_otsu(self.image)
        self.bw = closing(self.image > self.thresh, square(3))
        self.bw_clear = clear_border(self.bw)
        self.bw_label = label(self.bw_clear)
        self.regions = regionprops(self.bw_label)
#         self.idx = 0
        
    def plot_image_bw(self):
        plt.imshow(self.bw, cmap = plt.cm.bone)
    
    def plot_image(self):
        plt.imshow(self.bw_clear, cmap=plt.cm.bone)
        
    def plot_image_with_labels(self):
        plt.imshow(self.bw_label, cmap=plt.cm.bone)
        
    def max_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return max(lista)
        else:
            return -1
    
    def eccentricity(self):
        
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return self.regions[idx].eccentricity
        else:
            return -1
    
    def euler_number(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return self.regions[idx].euler_number
        else:
            return -1
    
    def solidity(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return self.regions[idx].solidity
        else:
            return -1
    
    def perimeter(self):
        lista = []
        for e in self.regions:
            lista.append(e.perimeter)
        if len(lista)>=1:
            return max(lista)
        else:
            return -1
    
    def mean_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            return statistics.mean(lista)
        else:
            return -1
    
    def std_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>2:
            return statistics.stdev(lista)
        else:
            return -1
    
    def thresh_img(self):
        return self.thresh
        

### Moments

In [7]:
class moments:
    
    def __init__(self, image):
        self.image = image
        self.moment = cv2.moments(self.image)
        self.hu = cv2.HuMoments(self.moment)
        
    def get_moments(self):
#         keys = [key for key in self.moment.keys()]
        values = [value for value in self.moment.values()]
        return values
    
    def get_HuMoments(self):
        moments_hu = []
        for m in range(len(self.hu)):
            moments_hu.append(self.hu[m][0])
        return moments_hu

#### DataFrame dcm_1

In [8]:
dcm_7 = load_pickle("dcm-7")
pat_7 = load_pickle("pat-7")

In [9]:
len(dcm_7), len(pat_7)

(2684, 2684)

In [10]:
data_dcm_7 = dict()
for dc in tqdm(range(len(dcm_7))):
    data_dcm_7[pat_7[dc]] = [s_entropy(dcm_7[dc]), entropy_simple(dcm_7[dc]), moments(dcm_7[dc]).get_moments(), moments(dcm_7[dc]).get_HuMoments(),
                      region_props(dcm_7[dc]).max_area(), region_props(dcm_7[dc]).eccentricity(), region_props(dcm_7[dc]).euler_number(),
                      region_props(dcm_7[dc]).solidity(), region_props(dcm_7[dc]).perimeter(), region_props(dcm_7[dc]).mean_area(), 
                      region_props(dcm_7[dc]).std_area(), region_props(dcm_7[dc]).thresh_img(), glcm(dcm_7[dc]).correlation(), glcm(dcm_7[dc]).homogeneity(),
                      glcm(dcm_7[dc]).contrast(), glcm(dcm_7[dc]).energy()]

100%|██████████| 2684/2684 [34:45<00:00,  1.37it/s]


In [11]:
df_dcm_7 = pd.DataFrame(data_dcm_7).T

In [12]:
df_dcm_7.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
79719bdf-3066-42d3-8496-6a12e12d2523,5.93957,4.117,"[101216151.0, 52780487321.0, 46436681299.0, 32...","[0.0010338282766687878, 5.939289345869184e-09,...",588028,0.281642,-10,0.882085,5960.72,49455,169610,86,"[0.9973229942183403, 0.9950963962272057, 0.997...","[0.6790586992781555, 0.6351255013914723, 0.722...","[34.60019359420822, 63.35128505755701, 33.3886...","[0.3121761474438403, 0.31040696024844294, 0.31..."
a9dbe0c0-3f75-4917-b44a-928692f5add0,7.794,5.40239,"[128028926.0, 67600650786.0, 70062476772.0, 44...","[0.0012706935285872695, 3.4503703473448275e-08...",507,0.740138,1,0.663613,143.912,11,56.3826,119,"[0.9979834276388474, 0.9966209506137944, 0.998...","[0.44481667252579404, 0.3695265339406694, 0.46...","[14.929468895100193, 24.97946354090523, 11.922...","[0.028834102176244856, 0.02614094688262479, 0...."
556ea435-2b79-45ca-9612-cde5f6bdffdc,7.76105,5.37955,"[125530750.0, 64975911689.0, 73998415784.0, 41...","[0.0011727763040181556, 3.764227418874079e-08,...",23222,0.977248,-10,0.305075,2897.64,221,2194.38,130,"[0.99906600061525, 0.9981286318683826, 0.99889...","[0.4721147271397937, 0.38854173542734194, 0.47...","[10.057540818976051, 20.130720696703115, 11.90...","[0.03761600661285582, 0.03428480403369405, 0.0..."
391589be-c014-4862-ae43-84f42b030f30,7.42821,5.14884,"[116298072.0, 58932072091.0, 69685037557.0, 38...","[0.0013033873541338438, 6.45571123845736e-09, ...",9652,0.920558,-2,0.441033,1155.46,170,1005.4,114,"[0.9943036165731565, 0.9893630345388245, 0.994...","[0.6268375642618058, 0.5552460248891534, 0.653...","[31.693947412634405, 59.13513624562721, 33.140...","[0.04989238107935342, 0.045713506116506056, 0...."
45cfd6b0-6dda-4193-8035-1cfc7826dcc7,7.2965,5.05755,"[125322934.0, 59807150228.0, 73561288163.0, 37...","[0.0012392127327575362, 4.97159257844427e-09, ...",566,0.968027,1,0.833579,159.113,16,67.7569,93,"[0.9946665851749104, 0.9905575038566216, 0.995...","[0.6446331002614172, 0.5647487226801587, 0.651...","[30.10355094544234, 53.23325201690542, 27.1629...","[0.061177725938090885, 0.057728981615845824, 0..."


In [13]:
df_dcm_7.to_pickle('df_dcm_7')

In [14]:
df_dcm_7.to_csv('df_dcm_7.csv')